In [1]:
%load_ext autoreload
%autoreload 2
import TRITON_SWMM_toolkit.examples as examples
import TRITON_SWMM_toolkit.analysis as analysis
from TRITON_SWMM_toolkit.process_simulation import TRITONSWMM_sim_post_processing
import TRITON_SWMM_toolkit.process_simulation as ps
import TRITON_SWMM_toolkit.processing_analysis as pe
from TRITON_SWMM_toolkit.process_simulation import *
import TRITON_SWMM_toolkit.plot_utils as plot_utils
import TRITON_SWMM_toolkit.utils as utils 
import TRITON_SWMM_toolkit.run_simulation as run
import TRITON_SWMM_toolkit.sensitivity_analysis as snst
from TRITON_SWMM_toolkit.config import analysis_config
from TRITON_SWMM_toolkit.examples import GetTS_TestCases as tst
from importlib import reload
reload(examples)
reload(analysis)
reload(ps)
reload(pe)
reload(plot_utils)
reload(utils)
reload(snst)
# norfolk_singlesim = examples.GetTS_TestCases.retreive_norfolk_single_sim_test_case(download_if_exists=False)
# norfolk_multisim = examples.GetTS_TestCases.retreive_norfolk_multi_sim_test_case()

rebuild_case = True
nrflk_multisim_ensemble = tst.retreive_norfolk_multi_sim_test_case(start_from_scratch=rebuild_case)
self = nrflk_multisim_ensemble.system.analysis
nrflk_multisim_ensemble.system.process_system_level_inputs(overwrite_if_exists=rebuild_case)
self.compile_TRITON_SWMM(recompile_if_already_done_successfully=rebuild_case)
# self.sensitivity.prepare_scenarios_in_each_subanalysis()
# self.prepare_all_scenarios()

In [2]:
nrflk_multisim_ensemble.system.sys_paths.print_all_paths()

/home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/
├── elevation.dem
└── mannings.dem


In [3]:
self.run_prepare_scenarios_serially(
        overwrite_scenarios=True, rerun_swmm_hydro_if_outputs_exist=True
    )

In [4]:
# self._update_log()
self.log.all_scenarios_created.get()


True

In [5]:
scenario = self.scenarios[0]
print(scenario.log.model_dump_json(indent = 2))

{
  "logfile": "/home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/sims/0-event_id.0/log.json",
  "event_iloc": 0,
  "event_idx": {
    "event_id": 0
  },
  "simulation_folder": "/home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/sims/0-event_id.0",
  "swmm_rainfall_dat_files": {},
  "storm_tide_for_swmm": null,
  "scenario_creation_complete": null,
  "inp_hydraulics_model_created_successfully": null,
  "inp_full_model_created_successfully": null,
  "inp_hydro_model_created_successfully": null,
  "hydro_swmm_sim_completed": null,
  "extbc_tseries_created": null,
  "extbc_loc_created": null,
  "hyg_timeseries_created": null,
  "hyg_locs_created": null,
  "inflow_nodes_in_hydraulic_inp_assigned": null,
  "triton_swmm_cfg_created": null,
  "sim_tritonswmm_executable_copied": null,
  "simulation_completed": false,
  "sim_log": {
    "run_attempts": {}
  },
  "TRITON_timeseries_written": false,
  "SWMM_node_timeseries